In [1]:
# Spam Detection Example

# Define the prior probabilities
P_A = 0.1 # probability of email being spam
P_B_given_A = 0.9 # probability of email containing "viagra" given it's spam
P_B_given_not_A = 0.01 # probability of email containing "viagra" given it's not spam
P_B = 0.05 # probability of email containing "viagra"

# Calculate the conditional probability of email being spam given it contains "viagra"
P_not_A = 1 - P_A # probability of email not being spam
P_A_given_B = P_B_given_A * P_A / (P_B_given_A * P_A + P_B_given_not_A * P_not_A)

print(P_A_given_B)


0.9090909090909092


In [4]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

# Hyperparameters
num_epochs = 5
batch_size = 128
learning_rate = 0.001

# Dataset
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, transform=transforms.ToTensor())

# Dataloader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

# Autoencoder model
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(784, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 12),
            nn.ReLU(),
            nn.Linear(12, 2),
        )
        self.decoder = nn.Sequential(
            nn.Linear(2, 12),
            nn.ReLU(),
            nn.Linear(12, 64),
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Linear(128, 784),
            nn.Sigmoid(),
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# Loss function and optimizer
model = Autoencoder()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, _) in enumerate(train_loader):
        # Flatten the image
        images = images.reshape(-1, 28*28)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, images)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print progress every 100 batches
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

# Testing loop
with torch.no_grad():
    for images, _ in test_loader:
        # Flatten the image
        images = images.reshape(-1, 28*28)

        # Forward pass
        outputs = model(images)

        # Compare original and reconstructed images
        mse_loss = nn.MSELoss(reduction='none')
        loss = mse_loss(outputs, images)
        psnr = 10 * torch.log10(1/loss.mean())
        print('PSNR: {:.2f}'.format(psnr))


    

100%|██████████| 9912422/9912422 [00:00<00:00, 54103550.73it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 3059445.72it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 21672369.16it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 13359417.09it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

Epoch [1/5], Step [100/469], Loss: 0.0691
Epoch [1/5], Step [200/469], Loss: 0.0679
Epoch [1/5], Step [300/469], Loss: 0.0618
Epoch [1/5], Step [400/469], Loss: 0.0581
Epoch [2/5], Step [100/469], Loss: 0.0522
Epoch [2/5], Step [200/469], Loss: 0.0513
Epoch [2/5], Step [300/469], Loss: 0.0504
Epoch [2/5], Step [400/469], Loss: 0.0501
Epoch [3/5], Step [100/469], Loss: 0.0454
Epoch [3/5], Step [200/469], Loss: 0.0473
Epoch [3/5], Step [300/469], Loss: 0.0476
Epoch [3/5], Step [400/469], Loss: 0.0454
Epoch [4/5], Step [100/469], Loss: 0.0436
Epoch [4/5], Step [200/469], Loss: 0.0426
Epoch [4/5], Step [300/469], Loss: 0.0437
Epoch [4/5], Step [400/469], Loss: 0.0456
Epoch [5/5], Step [100/469], Loss: 0.0482
Epoch [5/5], Step [200/469], Loss: 0.0409
Epoch [5/5], Step [300/469], Loss: 0.0443
Epoch [5/5], Step [400/469], Loss: 0.0461
PSNR: 13.67
PSNR: 13.85
PSNR: 13.67
PSNR: 13.53
PSNR: 13.37
PSNR: 13.61
PSNR: 13.60
P